# 고객요청의도 후 활용하기 (하수도 관련 문의 - 요금, 검침) - Validation test
## 작성자 : AISchool ( http://aischool.ai/%ec%98%a8%eb%9d%bc%ec%9d%b8-%ea%b0%95%ec%9d%98-%ec%b9%b4%ed%85%8c%ea%b3%a0%eb%a6%ac/ )
## 민원(콜센터) 질의-응답 데이터 다운받기 : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=98

# LangChain 라이브러리 설치

In [ ]:
!pip install langchain openai chromadb tiktoken pypdf unstructured sentence-transformers langchain-community

In [ ]:
import pandas as pd

# 데이터 읽어오기

In [ ]:
json_file_path = '/content/민원(콜센터) 질의응답_다산콜센터_생활하수도 관련 문의_Training.json'
df = pd.read_json(json_file_path)
df

In [ ]:
len(df)

In [ ]:
df.columns

# '고객의도' 종류 확인하기

In [ ]:
고객의도_리스트 = df['고객의도'].unique()
for idx, 고객의도 in enumerate(고객의도_리스트):
  print(idx, 고객의도)

# 고객의도가 '요금'과 관련된 데이터만 필터링하기

In [ ]:
요금_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '요금' in 의도]
요금_키워드_포함_고객의도_리스트

In [ ]:
filtered_df = df[(df['고객의도'] == '수도요금조회') & (df['문장번호'] == 1)]
filtered_df

# 고객의도가 '검침'과 관련된 데이터만 필터링하기

In [ ]:
검침_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '검침' in 의도]
검침_키워드_포함_고객의도_리스트

In [ ]:
filtered_df = df[(df['고객의도'] == '자가 수도검침') & (df['문장번호'] == 1)]
filtered_df

# 사용자 질문으로부터 '고객요청의도' 추출하기

##  OpenAI API Key 설정

In [ ]:
OPENAI_KEY = "여러분의_OPENAI_API_KEY"

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=OPENAI_KEY)

# create_tagging_chain으로 고객 요청의도 추출하기

In [ ]:
from langchain.chains import create_tagging_chain

# 고객요청의도 추출을 위한 schema 설정
schema = {
    "properties": {
        "고객요청의도": {
            "type": "string",
            "enum": ['수도요금',
                     '검침'
            ]

        }
    }
}
extraction_chain = create_tagging_chain(schema, llm)

In [ ]:
chain_run_result = extraction_chain.run("수도요금 문의하고 싶습니다.")
chain_run_result

In [ ]:
chain_run_result = extraction_chain.run("수도검침을 본인이하면 요금 감면이 되나요?")
chain_run_result

In [ ]:
chain_run_result = extraction_chain.run("할인도 되나요?")
chain_run_result

# Vectorstore에 저장하기 위한 데이터 정제하기

In [ ]:
요금_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '요금' in 의도]
요금_키워드_포함_고객의도_리스트

In [ ]:
검침_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '검침' in 의도]
검침_키워드_포함_고객의도_리스트

In [ ]:
최종_필터링용_고객의도_리스트 = 요금_키워드_포함_고객의도_리스트 + 검침_키워드_포함_고객의도_리스트
최종_필터링용_고객의도_리스트

In [ ]:
len(최종_필터링용_고객의도_리스트)

In [ ]:
filtered_df = df[df['고객의도'].isin(최종_필터링용_고객의도_리스트)]
filtered_df

# 중복 데이터 제거

In [ ]:
# Identify duplicates by considering all columns except '대화셋일련번호'
duplicates = filtered_df.duplicated(subset=filtered_df.columns.difference(['대화셋일련번호']), keep=False)

# Remove duplicates
df_cleaned = filtered_df[~duplicates]
df_cleaned

In [ ]:
대화셋_set = set()
for count, (index, row) in enumerate(df_cleaned.iterrows()):
    print(count, row['대화셋일련번호'], row['고객질문(요청)'])
    대화셋_set.add(row['대화셋일련번호'])

In [ ]:
대화셋_set

In [ ]:
filtered_df = df[df['대화셋일련번호'].isin(대화셋_set)]
filtered_df

In [ ]:
# '대화셋일련번호'로 그룹화하고 각 그룹을 인덱스 순서대로 정렬하여 원하는 컬럼들을 하나의 문자열로 합치기
def combine_columns(group):
    sorted_group = group.sort_index()  # 인덱스 순서대로 정렬
    combined_string = " ".join(
        sorted_group[['고객질문(요청)', '상담사질문(요청)', '고객답변', '상담사답변']].astype(str).values.flatten()
    )

    # 첫번째 '고객의도' 값 가져오기
    customer_intent = sorted_group['고객의도'].iloc[0]

    # 만약 '고객의도'가 없을 경우 '상담사의도'에서 값 가져오기
    if pd.isna(customer_intent) or customer_intent == '':
        customer_intent = sorted_group['상담사의도'].iloc[0]

    return pd.Series({'combined_string': combined_string, '고객의도': customer_intent})

In [ ]:
combined_result = filtered_df.groupby('대화셋일련번호').apply(combine_columns).reset_index()
combined_result

In [ ]:
# '고객요청의도' 컬럼 생성
def assign_customer_request_intent(intent):
    if intent in 요금_키워드_포함_고객의도_리스트:
        return '수도요금'
    elif intent in 검침_키워드_포함_고객의도_리스트:
        return '검침'

combined_result['고객요청의도'] = combined_result['고객의도'].apply(assign_customer_request_intent)
combined_result

# 중간에 고객의도 바뀌는 데이터 삭제

In [ ]:
combined_result[combined_result['대화셋일련번호']=='B23720']

In [ ]:
temp_df = df[df['대화셋일련번호'] == 'B23720']
temp_df

In [ ]:
combined_result = combined_result.dropna(subset=['고객요청의도'])
combined_result

In [ ]:
combined_result[combined_result['대화셋일련번호']=='B10715']['combined_string'].values[0]

In [ ]:
# 각 row의 값들을 순회하면서 출력
for index, row in combined_result.iterrows():
    print(f"대화셋일련번호: {row['대화셋일련번호']}, 고객요청의도: {row['고객요청의도']},  Combined String: {row['combined_string']}")

# Document 생성 및 Vectorstore에 Embedding해서 저장

In [ ]:
from langchain.schema import Document

docs = []
수도요금_docs = []
검침_docs = []

# 각 row의 값들을 순회하면서 출력
for index, row in combined_result.iterrows():

    doc = Document(page_content=row['combined_string'], metadata={'대화셋일련번호': row['대화셋일련번호'], '고객요청의도': row['고객요청의도']})
    docs.append(doc)

    if row['고객요청의도'] == '수도요금':
        수도요금_docs.append(doc)
    elif row['고객요청의도'] == '검침':
        검침_docs.append(doc)

In [ ]:
docs

In [ ]:
len(docs)

In [ ]:
수도요금_docs

In [ ]:
len(수도요금_docs)

In [ ]:
검침_docs

In [ ]:
len(검침_docs)

# Embedding

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sroberta-multitask" # (KorNLU 데이터셋에 학습시킨 한국어 임베딩 모델)
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=docs, embedding=embedding_model, collection_name="all")

In [ ]:
수도요금_vectorstore = Chroma.from_documents(documents=수도요금_docs, embedding=embedding_model, collection_name="waterbill")

In [ ]:
검침_vectorstore = Chroma.from_documents(documents=검침_docs, embedding=embedding_model, collection_name="meterreading")

In [ ]:
len(vectorstore)

In [ ]:
len(수도요금_vectorstore)

In [ ]:
len(검침_vectorstore)

# Sanity Check

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
수도요금_retriever = 수도요금_vectorstore.as_retriever(search_kwargs={"k": 1})
검침_retriever = 검침_vectorstore.as_retriever(search_kwargs={"k": 1})

In [ ]:
retrieved_docs = retriever.invoke(
    "다자녀는 수도 요금 할인되나요?"
)
print(retrieved_docs[0].page_content)

In [ ]:
retrieved_docs = 수도요금_retriever.invoke(
    "다자녀는 수도 요금 할인되나요?"
)
print(retrieved_docs[0].page_content)

In [ ]:
retrieved_docs = 검침_retriever.invoke(
    "다자녀는 수도 요금 할인되나요?"
)
print(retrieved_docs[0].page_content)

In [ ]:
retrieved_docs = retriever.invoke(
    "수도도 자가검침이 가능한가요?"
)
print(retrieved_docs[0].page_content)

In [ ]:
retrieved_docs = 수도요금_retriever.invoke(
    "수도도 자가검침이 가능한가요?"
)
print(retrieved_docs[0].page_content)

In [ ]:
retrieved_docs = 검침_retriever.invoke(
    "수도도 자가검침이 가능한가요?"
)
print(retrieved_docs[0].page_content)

# Multiple Retriever를 활용하여 연관된 문서 찾아오기

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=OPENAI_KEY)

In [ ]:
retrievers = {
    "": retriever,
    "수도요금": 수도요금_retriever,
    "검침": 검침_retriever,
}

In [ ]:
from langchain_core.runnables import chain

In [ ]:
@chain
def custom_chain(question):
    response = extraction_chain.run(question)
    #print(response.get('고객요청의도', ""))
    retriever = retrievers[response.get('고객요청의도') or ""]
    return retriever.invoke(question)

In [ ]:
custom_chain.invoke("다자녀는 수도 요금 할인되나요?")

In [ ]:
custom_chain.invoke("수도도 자가검침이 가능한가요?")

In [ ]:
custom_chain.invoke("할인도 되나요?")

# 대화 history 저장을 위한 ConversationBufferMemory 설정

**ConversationBufferMemory** : 모든 대화내역을 저장

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

In [ ]:
memory.load_memory_variables({})

In [ ]:
def load_memory(input):
    return memory.load_memory_variables({})["history"]

# Tracking을 위한 LangSmith 설정(Optional)

In [ ]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"User Intent Analysis (sewer) (validation test) - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "여러분의_LangSmith_API_key"

In [ ]:
unique_id

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
def using_intent_get_relevant_document(user_question):
    histroy_combined_question =  memory.load_memory_variables({})["history"] + '\nHuman::' + user_question

    relevant_document = custom_chain.invoke(histroy_combined_question)[0]

    # print(histroy_combined_question)
    # print(relevant_document)
    return relevant_document

In [ ]:
relevant_document = using_intent_get_relevant_document("다자녀는 수도 요금 할인되나요?")
relevant_document

# 최종 Prompt 및 Chain 생성

In [ ]:
from langchain.prompts import PromptTemplate

template = """다음과 같은 맥락과 채팅히스토리를 사용하여 마지막 질문에 대답하십시오.
맥락: {context}
채팅히스토리 : {history}
질문: {question}
도움이 되는 답변:"""
rag_prompt_custom = PromptTemplate.from_template(template)

In [ ]:
# RAG chain 설정
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import MessagesPlaceholder

rag_chain = {"context": using_intent_get_relevant_document, "question": RunnablePassthrough(), "history": load_memory} | rag_prompt_custom | llm

In [ ]:
def get_result_and_save_memory(user_question):
  chat_result = rag_chain.invoke(user_question)

  # memory에 저장
  memory.chat_memory.add_user_message(user_question)
  memory.chat_memory.add_ai_message(chat_result.content)

  return chat_result.content

# 수도요금, 검침 AI 고객센터 챗봇 성능 evaluation (정성적 평가) - Validation 데이터에 테스트

In [ ]:
json_file_path = '/content/민원(콜센터) 질의응답_다산콜센터_생활하수도 관련 문의_Validation.json'
validation_df = pd.read_json(json_file_path)
validation_df

In [ ]:
validation_고객의도_리스트 = validation_df['고객의도'].unique()
for idx, 고객의도 in enumerate(validation_고객의도_리스트):
  print(idx, 고객의도)

In [ ]:
요금_키워드_포함_고객의도_리스트 = [의도 for 의도 in validation_고객의도_리스트 if '요금' in 의도]
요금_키워드_포함_고객의도_리스트

In [ ]:
검침_키워드_포함_고객의도_리스트 = [의도 for 의도 in validation_고객의도_리스트 if '검침' in 의도]
검침_키워드_포함_고객의도_리스트

In [ ]:
최종_필터링용_고객의도_리스트 = 요금_키워드_포함_고객의도_리스트 + 검침_키워드_포함_고객의도_리스트
최종_필터링용_고객의도_리스트

In [ ]:
len(최종_필터링용_고객의도_리스트)

In [ ]:
filtered_df = validation_df[validation_df['고객의도'].isin(최종_필터링용_고객의도_리스트)]
filtered_df

# 중복 데이터 제거

In [ ]:
# Identify duplicates by considering all columns except '대화셋일련번호'
duplicates = filtered_df.duplicated(subset=filtered_df.columns.difference(['대화셋일련번호']), keep=False)

# Remove duplicates
df_cleaned = filtered_df[~duplicates]
df_cleaned

In [ ]:
대화셋_set = set()
for count, (index, row) in enumerate(df_cleaned.iterrows()):
    print(count, row['대화셋일련번호'], row['고객질문(요청)'])
    대화셋_set.add(row['대화셋일련번호'])

In [ ]:
대화셋_set

In [ ]:
len(대화셋_set)

## Case 1 - 검침

In [ ]:
# 대화내역을 저장한 Memory 초기화하기
memory.clear()

In [ ]:
temp_df = validation_df[validation_df['대화셋일련번호'] == 'B41393']
temp_df

In [ ]:
llm_response = get_result_and_save_memory("자가검침 관련해서 여쭙고 싶습니다.")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("수도요금 할인은 무슨 말인가요?")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("아 그럼 저도 신청해서 할 수 있는 건가요?")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("신청 하는 방법을 알 수 있을까요?")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("검침 방법도 설멸 주시는 거지요/")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("셍각보다 쉬워 보이는 군요.")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("고맙습니다. 혹시라도 주의해야 되는 사항이 있나요?")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("그래도 혹시 오류가 있거나 일부러 낮게 기재하는 경우가 있지 않을까요?")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("안심이 되네요. 신청해 봐야겠습니다.")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("없습니다. 확인해보고 신청해 봐야 겠네요.")
llm_response

In [ ]:

memory.load_memory_variables({})["history"]

## Case 2 - 수도요금

In [ ]:
# 대화내역을 저장한 Memory 초기화하기
memory.clear()

In [ ]:
temp_df = validation_df[validation_df['대화셋일련번호'] == 'B36668']
temp_df

In [ ]:
llm_response = get_result_and_save_memory("공동 수도요금은 어떻게 계산되는 거예요?")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("그럼 주계량기 사용량이 공동 수도요금 인가요?")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("공동 사용량을 세대별로 나눠서 부과 하는 건가요?")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("수도요금에 같이 고지되어 나오는거 맞죠?")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("공동주택은 다 그런가요?")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]

In [ ]:
llm_response = get_result_and_save_memory("그렇구나, 알겠습니다.")
llm_response

In [ ]:
memory.load_memory_variables({})["history"]